In [3]:
import pandas as pd

# Load the book chapters to be uploaded
book_chapters = pd.read_csv('data/book_chapters_to_upload.csv', na_filter=False, dtype = str)

# Test with the first 20 book chapter rows
#book_chapters = book_chapters.head(20)

# Load the works already uploaded
works = pd.read_csv('data/works.csv', na_filter=False, dtype = str)

# Create a copy of the book chapters dataframe to remove the rows that are already uploaded
book_chapters_to_upload = book_chapters.copy()

# For each book chapter, check if the work is already uploaded by comparing the Key column in the book chapters to the unique_identifier column in the works
for index, row in book_chapters.iterrows():
    print(row['Key'])
    if row['Key'] in works['unique_identifier'].values:
        # If the work is already uploaded, remove the row from the book chapters to be uploaded
        book_chapters_to_upload = book_chapters_to_upload[book_chapters_to_upload.Key != row['Key']]
    
# Save the book chapters to be uploaded to a CSV file
book_chapters_to_upload.to_csv('data/book_chapters_to_upload_new.csv', index=False)




IA4WAMRM
W8GIKFFC
ZEFXJN83
5F47ID3V
T3WK6Q53
2HGCKK5X
6AZXXUXE
PWHV3RUD
8NIF8I9X
3584NUVQ
9P4CK4VM
HKIWGHU7
LWTPSECW
HPDTRZPP
BI6EE3IK
U57MP977
S3K74EHC
NMLDGYB2
MNXDLDYN
DQSHVZXY
F92C9HGA
X6DZKIW6
G2UI32K3
ZCPZQXT5
LNS27SN3
9P5SPK84
QJXKA2VA
BQ29ZB8B
FX7S592H
RJFPKF5W
K4IH7VF2
WHBTY3VF
JC29PXMR
Q5I5PGIT
25JMXR4P
M8T6RPW2
SAJVVHNX
3Q8U5QAQ
RH2CQWKN
QMR4VWZK
ZRR4U4KU
ZRR4U4KU
RSP52X4D
TH75UNCB
528VN46J
NTFCYFN8
GVDFV2XX
P5I2DKWJ
F4UA6CWP
U8HPASEN
HDWUNSNH
9HB7CSXQ
PCAC73PQ
Z9ZDFVTJ
RRDLU37A
GVVPXTIV
AGFFAEHB
AA78HV45
CDCT7GFA
BHNVD9WS
XIB69ECU
NZM4TTN6
GZN64872
CEW297SQ
UGVVTABJ
J27GS326
6QXAPGXK
P8AQMD3N
R99XKJVJ
TFKACNVH
M2ISEFGV
663GDXPC
E64RPUBM
VKVNB8RP
3BNKSNKB
7UZE7SVW
9PPI42UJ
9WZP947T
3WN6CTCP
I8SBX3G8
9NZB98WQ
VA6ZUFHC
Q3E8WJEI
UGE4E8GK
4RUWKUFK
WSKI2VXD
4XFIZTPD
LH8XND6Q
BQ7YF8XC
RNWYRMQU
MMDCCFTC
Y3Q6KD6F
87UG8WWD
QY6MZ26Z
KX9HMFB3
F4ENCAXK
BSYUEBWI
UJV3ZUF2
A4EKPQTE
C7QGIFDN
J6ZAAYWC
SGUQ2E6P
AI9WVJR7
RF7VBC77
8ER4ISP7
G44BLD7A
5WVCRUGW
HUPBB6IU
2SN73SZT
D8N3D6X4
P3I3UW3D
M

In [ ]:
books = list(works['Publication Title'])[:10]
for book in books:
    print(book)
    hits = find_containing_book('', book, 'en', user_agent)
    print(hits)
    print()

# Identify books that may already be in Wikidata

In [25]:
import pandas as pd
import json
import yaml
import requests
import requests_cache
from typing import List, Dict, Tuple, Any, Optional
from fuzzywuzzy import fuzz # fuzzy logic matching

# Set up cache for HTTP requests
requests_cache.install_cache('wqs_cache', backend='sqlite', expire_after=300, allowable_methods=['GET', 'POST'])

def search_cirrus(search_string: str, http: requests.Session, api_endpoint='https://www.wikidata.org/w/api.php') -> List[dict]:
	"""Search for a string using CirrusSearch (Elastic-based search engine)

	Parameters
	----------
	search_string : str
		String to use in elastic search, produces same results as the Wikidata search box
	http : requests.Session
		Requests HTTP session to use for search calls
	api_endpoint : str
		URL of the endpoint, defaults to Wikidata
	
	Returns
	-------
	List[dict] : List of search results with keys "qid", "description", and "label"
	"""
	request_string = '''{
		"action": "query",
		"format": "json",
		"list": "search",
		"formatversion": "2",
		"srsearch": "''' + search_string + '''"
	}'''

	response = http.get(api_endpoint, params=json.loads(request_string))
	data = response.json()
	#print(json.dumps(data, indent=2))

	hits = data['query']['search']

	# Look up the label for each item
	for index, hit in enumerate(hits):
		# "title" in the search results is the Q ID		
		request_string = '''{
			"action": "wbgetentities",
			"format": "json",
			"ids": "''' + hit['title'] + '''",
			"props": "labels"
		}'''
		response = http.get(api_endpoint, params=json.loads(request_string))
		data = response.json()
		#print(json.dumps(data, indent=2))

		# Match the Q ID to the label and add the label to the hits list
		try:
			hits[index]['label'] = data['entities'][hit['title']]['labels']['en']['value']
		except:
			hits[index]['label'] = ''

	#print(json.dumps(hits, indent=2))

	# Clean up the hits list by removing useless keys and renaming others
	clean_hits = []
	for hit in hits:
		del hit['ns']
		del hit['pageid']
		del hit['size']
		del hit['wordcount']
		del hit['timestamp']
		hit['qid'] = hit['title']
		del hit['title']
		hit['description'] = hit['snippet']
		del hit['snippet']
		clean_hits.append(hit)

	return clean_hits

# Load program settings, mappings, screens, and configurations
with open('settings.yaml', 'r') as file_object:
    settings = yaml.safe_load(file_object)


# Set up the session for the Cirrus Search API

# The user_agent string identifies this application to Wikimedia APIs.
# If you modify this script, you need to modify the settings to change the user-agent string to something else!
user_agent = 'PubLoader/' + settings['script_version'] + ' (mailto:' + settings['operator_email_address'] + ')'

# Set up a session for performing Cirrus searches
request_header_dictionary = {
	'Accept' : 'application/json',
	'User-Agent': user_agent
}
SEARCH_SESSION = requests.Session()
# Set default User-Agent header so you don't have to send it with every request
SEARCH_SESSION.headers.update(request_header_dictionary)


# Load the book chapter data
book_chapters = pd.read_csv('data/book_chapters_to_upload.csv', na_filter=False, dtype = str)
print(len(book_chapters))

# Remove rows with duplicate Publication Title values to create dataframe containing only unique book titles
book_titles = book_chapters.drop_duplicates(subset=['Publication Title']).copy()
# Reset the index of the dataframe
book_titles = book_titles.reset_index(drop=True)

print(len(book_titles))

# Create a dictionary where the key is the chapter Key and the value is the index of the book in the book_list
book_chapter_to_book_index = {}
for index, row in book_chapters.iterrows():
	# Find the index of the book whose title matches the book chapter's Publication Title
    book_index = book_titles[book_titles['Publication Title'] == row['Publication Title']].index[0]
    book_chapter_to_book_index[row['Key']] = book_index


619
527
{'IA4WAMRM': 0, 'W8GIKFFC': 1, 'ZEFXJN83': 2, '5F47ID3V': 3, 'T3WK6Q53': 4, '2HGCKK5X': 5, '6AZXXUXE': 6, 'PWHV3RUD': 7, '8NIF8I9X': 8, '9P4CK4VM': 9, 'HKIWGHU7': 10, 'LWTPSECW': 11, 'HPDTRZPP': 12, 'S3K74EHC': 13, 'NMLDGYB2': 14, 'MNXDLDYN': 15, 'X6DZKIW6': 16, 'G2UI32K3': 17, 'ZCPZQXT5': 18, 'LNS27SN3': 19, '9P5SPK84': 20, 'QJXKA2VA': 20, 'BQ29ZB8B': 20, 'FX7S592H': 21, 'RJFPKF5W': 22, 'K4IH7VF2': 23, 'WHBTY3VF': 24, 'JC29PXMR': 25, 'Q5I5PGIT': 26, '25JMXR4P': 27, 'M8T6RPW2': 28, 'SAJVVHNX': 29, 'RH2CQWKN': 30, 'QMR4VWZK': 31, 'ZRR4U4KU': 32, 'RSP52X4D': 32, 'TH75UNCB': 33, '528VN46J': 34, 'NTFCYFN8': 35, 'GVDFV2XX': 36, 'P5I2DKWJ': 37, 'F4UA6CWP': 38, 'U8HPASEN': 39, 'HDWUNSNH': 40, '9HB7CSXQ': 41, 'PCAC73PQ': 42, 'Z9ZDFVTJ': 43, 'RRDLU37A': 44, 'GVVPXTIV': 45, 'AA78HV45': 46, 'CDCT7GFA': 47, 'BHNVD9WS': 47, 'NZM4TTN6': 48, 'GZN64872': 49, '6QXAPGXK': 50, 'P8AQMD3N': 51, 'R99XKJVJ': 52, 'TFKACNVH': 53, 'M2ISEFGV': 54, '663GDXPC': 55, 'E64RPUBM': 56, 'VKVNB8RP': 56, '3BNKSNKB

In [28]:
books = list(book_titles['Publication Title'])[:100]
for book in books:
    print(book)
    clean = book.replace('"', '')
    hits = search_cirrus(clean, SEARCH_SESSION)

    # Perform a fuzzy match with each hit
    for hit in hits:
        w_ratio = fuzz.WRatio(hit['label'], clean)

        # Test for nearly exact title match
        if w_ratio > settings['existing_work_fuzzy_match_cutoff']:
            # 

    print()

'Imagining' biblical worlds
[]

"A wise and discerning mind": essays in honor of Burke O. Long
[]

"Alle Verhältnisse umzuwerfen ... und die Mächtigen vom Thron zu stürzen": das gemeinsame Erbe von Christen und Marx
[]

"Diese Wirtschaft tötet": (Papst Franziskus) Kirchen gegen Kapitalismus
[]

"I studied inscriptions before the Flood"
[]

"Sha'Arei Talmon": Studies in the Bible, Qumran, and the Ancient Near East Presented to Shemaryahu Talmon
[]

A commentary on the general instruction of the Roman Missal : developed under the auspices of the Catholic Academy of Liturgy and cosponsored by the Federation of Diocesan Liturgical Commissions
[]

A Companion to Ancient Epic
[{'label': 'Literature (J.M.) Foley Ed. A Companion to Ancient Epic. Malden, MA and Oxford: Blackwell, 2005. Pp. xxv + 664, illus. £85. 9781405105248', 'qid': 'Q60209624', 'description': ''}]

A Critical Engagement: Essays on the Hebrew Bible in Honour of J. Cheryl Exum
[]

A Feminist Companion to Tobit and Judith
[]

A